In [2]:
import numpy as np
import pandas as pd
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression


In [3]:
corpus = Corpus(filename=download('supreme-corpus'))

Dataset already exists at /Users/fvescia/.convokit/downloads/supreme-corpus


In [4]:
# All cases
cases = pd.read_json(path_or_buf='data/cases.jsonl', lines=True)

# Cases with clear winners
df = cases.loc[cases.loc[:, 'win_side'].isin([1, 0])]

# Roberts court cases with clear winners
roberts = df.loc[df.loc[:, 'court'] == 'Roberts Court', :]

# All utterances
all_utts = corpus.get_utterances_dataframe()

# Roberts court case utterances
roberts_ids = roberts.loc[:, 'id'].unique()
utts = all_utts.loc[all_utts.loc[:, 'meta.case_id'].isin(roberts_ids)]

# Roberts court cases with clearn winners and utterance data
    # Unique case IDs from utts indicate the number of Roberts court cases
    # the corpus has utterance data for
subset_ids = utts.loc[:, 'meta.case_id'].unique()
    # Use these ids to subset the roberts data frame
    # (exclude cases without utterance data)
roberts = roberts.loc[roberts.loc[:, 'id'].isin(subset_ids)]
print(len(roberts), 'cases')
petitioner_wins = roberts.loc[:, 'win_side'].mean()
print(round(petitioner_wins * 100, 2),
      'percent of cases were decided favorably for the petitioner')

1024 cases
65.33 percent of cases were decided favorably for the petitioner


In [6]:
# Utterances per case
print('Min:', utts.groupby(['meta.case_id']).size().min()) 
print('Max:', utts.groupby(['meta.case_id']).size().max())
print('Mean', utts.groupby(['meta.case_id']).size().mean())

Min: 1
Max: 1235
Mean 237.3955078125


In [7]:
# Drop single-utterance cases
utt_counts = pd.DataFrame(utts.groupby(['meta.case_id']).size())
utt_counts = utt_counts.reset_index()
utt_counts = utt_counts.rename(columns={0: 'utt_counts'})
utts = pd.merge(utts, utt_counts, how = 'left')
utts = utts.loc[utts.loc[:, 'utt_counts'] != 1, :]
print('1 case dropped')
print('New min:', utts.groupby(['meta.case_id']).size().min()) 
print(len(utts.groupby(['meta.case_id'])), 'cases')

# Store utts for use in other notebooks
%store utts

1 case dropped
New min: 66
1023 cases
Stored 'utts' (DataFrame)


In [11]:
utts['justice'] = utts.loc[:, 'meta.speaker_type'] == 'J'
df = utts.groupby('meta.case_id').agg({'justice': ['sum', 'count']})
df.columns = [ 'justice_utts', 'tot_utts']
df['justice_utt_share'] = df.loc[:, 'justice_utts'] / df.loc[:, 'tot_utts']
justices = df.loc[:, ['justice_utt_share']]
print(justices)
print('Min:', justices.loc[:, 'justice_utt_share'].min())
print('Max:', justices.loc[:, 'justice_utt_share'].max())
print('Mean:', justices.loc[:, 'justice_utt_share'].mean())

advocates = utts.loc[utts.loc[:, 'meta.speaker_type'] == 'A', :].copy()
advocates['petitioner_advocate'] = advocates.loc[:, 'meta.side'] == 1
advocates = advocates.groupby('meta.case_id').agg({'petitioner_advocate': ['sum', 'count']})
advocates.columns = [ 'petitioner_advocate_utts', 'total_advocate_utts']
advocates['petitioner_advocate_utt_share'] = advocates.loc[:, 'petitioner_advocate_utts'] / advocates.loc[:, 'total_advocate_utts']
advocates = advocates.loc[:, [ 'petitioner_advocate_utt_share']]
print(advocates)
print('Min', advocates.loc[:, 'petitioner_advocate_utt_share'].min())
print('Max', advocates.loc[:, 'petitioner_advocate_utt_share'].max())
print('Mean', advocates.loc[:, 'petitioner_advocate_utt_share'].mean())
%store advocates

utt_shares = pd.merge(justices, advocates, how='left',left_index = True, right_index = True)
utt_shares

               justice_utt_share
meta.case_id                    
2005_03-1238            0.515571
2005_04-1034            0.513353
2005_04-10566           0.525333
2005_04-1067            0.534954
2005_04-1084            0.526611
...                          ...
2019_19-631             0.572193
2019_19-635             0.583916
2019_19-67              0.532710
2019_19-7               0.538793
2019_19-715             0.593750

[1023 rows x 1 columns]
Min: 0.48
Max: 0.7347826086956522
Mean: 0.5158600540883759
               petitioner_advocate_utt_share
meta.case_id                                
2005_03-1238                        1.000000
2005_04-1034                        0.603659
2005_04-10566                       0.533708
2005_04-1067                        0.483660
2005_04-1084                        0.514793
...                                      ...
2019_19-631                         0.450000
2019_19-635                         0.361345
2019_19-67                          0

,justice_utt_share,petitioner_advocate_utt_share
meta.case_id,,
2005_03-1238,0.515571,1.000000
2005_04-1034,0.513353,0.603659
2005_04-10566,0.525333,0.533708
2005_04-1067,0.534954,0.483660
2005_04-1084,0.526611,0.514793
...,...,...
2019_19-631,0.572193,0.450000
2019_19-635,0.583916,0.361345
2019_19-67,0.532710,0.540000


In [7]:
utts['text'].replace('\d+', '', regex=True, inplace = True) #remove numbers, decide as group if this makes sense
convo_case = pd.DataFrame(utts.groupby('meta.case_id')['conversation_id'].nunique())
convo_case.rename(columns = {'conversation_id':'convo_count'}, inplace = True)
#Code to create conversations/case feature

group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
#NOTE: No stemming or lemmatization done at this point
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != 'win_side'],df['win_side'], test_size = 0.2, random_state = 0)

In [9]:
#Initialize Vectorizer and vectorize train data
count_vect = CountVectorizer(ngram_range = (1,1),min_df = 4,
            stop_words = 'english', token_pattern = r'\b[a-zA-Z]{3,}\b') 
count_df = count_vect.fit_transform(X_train['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = count_vect.get_feature_names_out(), index = X_train['id'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(count_df).toarray()
X_train_tfidf = pd.DataFrame(X_train_tfidf, columns = count_vect.get_feature_names_out(), index = X_train['id'])

X_train_tfidf = pd.merge(X_train_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
X_train_tfidf = pd.merge(X_train_tfidf,utt_shares, how = 'left', left_index = True, right_index = True)
#MERGING IN ADDITIONAL FEATURES HERE
X_train_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abatement,...,zieve,zillion,zip,zipes,zone,zones,zoning,convo_count,justice_utt_share,petitioner_advocate_utt_share
id,,,,,,,,,,,,,,,,,,,,,
2015_15-290,0.000000,0.000000,0.007648,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.532934,0.423077
2007_07-343,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.515982,0.584906
2009_09-497,0.006649,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.512727,0.425373
2005_04-1131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.011321,0.006974,0.0,0.0,1,0.509434,0.584615
2009_09-338,0.000000,0.008387,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.519573,0.518519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016_16-6219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.515982,0.462264
2007_06-984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.516279,0.341346
2013_12-79,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1,0.502674,0.354839


In [43]:
#Vectorize test data
count_test = count_vect.transform(X_test['text'])
count_test_array = count_test.toarray()
count_test_df = pd.DataFrame(count_test_array, columns = count_vect.get_feature_names(),index = X_test['id'])

X_test_tfidf = tfidf_transformer.transform(count_test_df).toarray()
X_test_tfidf = pd.DataFrame(X_test_tfidf, columns = count_vect.get_feature_names(), index = X_test['id'])
X_test_tfidf = pd.merge(X_test_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
X_test_tfidf = pd.merge(X_test_tfidf,utt_shares, how = 'left', left_index = True, right_index = True)
X_test_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abatement,...,zieve,zillion,zip,zipes,zone,zones,zoning,convo_count,justice_utt_share,petitioner_advocate_utt_share
id,,,,,,,,,,,,,,,,,,,,,
2016_15-649,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.512048,0.222222
2005_04-1544,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.502110,0.483051
2006_05-1074,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.513514,0.537037
2013_12-158,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.004829,0.0,0.0,1,0.520958,0.475000
2009_08-1553,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.503425,0.296552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015_14-419,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.524887,0.466667
2009_08-1529,0.0,0.0,0.0,0.005098,0.021389,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.505814,0.235294
2009_09-475,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1,0.506024,0.463415


In [44]:

logit = LogisticRegression()
logit = logit.fit(X = X_train_tfidf, y = y_train)
logit.predict(X_test_tfidf)
mean_acc = logit.score(X_test_tfidf, y_test)
print(mean_acc)
"""
X_test_tfidf AND X_train_tfidf ARE OUR CURRENT BAG OF WORDS FEATURE SETS.
"""
count_df.to_csv('data/train_unigram_BoW.csv')
count_test_df.to_csv('data/test_unigram_BoW.csv')

0.6634146341463415


In [35]:
np.mean(y_test)

0.6292682926829268